In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import random
from datetime import datetime

In [ ]:
# Params
p_duplicate = 40 # in percent (limit to 100)
p_distribution_altered = 40
p_distribution_aterm = 60

columns_drop = ['patient', 'video', 'param']
# columns_drop = ['patient', 'video', 'param', '15_seconds_after', 'stimul']

In [ ]:
if p_distribution_altered + p_distribution_aterm != 100:
    raise Exception('Fail distribution new dataset!')

In [ ]:
original_dataset = pd.read_csv('data_final.csv')

In [ ]:
print(original_dataset.head())

In [ ]:
altered_list = original_dataset['classify'] != 'Atermo'
altered_dataset = original_dataset[altered_list]
# print(altered_dataset.head())

In [ ]:
aterm_list_1 = original_dataset['classify'] == 'Atermo'
aterm_list_2 = original_dataset['6_seconds_after'] > 0
aterm_list_3 = aterm_list_1 == aterm_list_2
aterm_dataset = original_dataset[aterm_list_3]
# print(aterm_dataset.head())

In [ ]:
register_altered = altered_dataset.shape[0]
register_aterm = aterm_dataset.shape[0]
print(register_altered, register_aterm)

In [ ]:
total = register_altered + register_aterm
print("%d (%.2f%%) for altered and %d (%.2f%%) for aterm" % (register_altered,
                                                             (register_altered/total)*100,
                                                             register_aterm,
                                                             (register_aterm/total)*100))

In [ ]:
register_altered_new =  register_altered + (register_altered * (p_duplicate / 100))
register_aterm_new = int(register_altered_new / 0.6)
total = register_altered_new + register_aterm_new
print("%d (%.2f%%) for altered and %d (%.2f%%) for aterm" % (register_altered_new,
                                                             (register_altered_new/total)*100,
                                                             register_aterm_new,
                                                             (register_aterm_new/total)*100))

In [ ]:
# Creating a seed for the random process based on real time
seed = datetime.now()
y, m, d, h, M, s = datetime.now().year, datetime.now().month, datetime.now().day, datetime.now().hour, datetime.now().minute, datetime.now().second
random.seed(y+m+d*h-(M*s))

In [ ]:
print('Duplicate altered registers, original %d for %d' % (altered_dataset.shape[0], register_altered_new))
if altered_dataset.shape[0] < register_altered_new:
    new_altered_dataset = altered_dataset
    while(new_altered_dataset.shape[0] < register_altered_new-1):
        aux = random.randint(0, altered_dataset.shape[0]-1)
        new_altered_dataset = new_altered_dataset.append(altered_dataset.iloc[aux])
        
    print('Finish new altered dataset, %d register' % (new_altered_dataset.shape[0]))
else:
    new_altered_dataset = altered_dataset

In [ ]:
print('Duplicate aterms registers, original %d for %d' % (aterm_dataset.shape[0], register_aterm_new))
if aterm_dataset.shape[0] < register_aterm_new:    
    new_aterm_dataset = aterm_dataset
    while(new_aterm_dataset.shape[0] < register_aterm_new-1):
        aux = random.randint(0, aterm_dataset.shape[0]-1)
        new_aterm_dataset = new_aterm_dataset.append(aterm_dataset.iloc[aux])
        
    print('Finish new aterm dataset, %d register' % (new_altered_dataset.shape[0]))
else:
    new_aterm_dataset = aterm_dataset

In [ ]:
list_index_insert_altered = [index for index, item in new_altered_dataset.iterrows()]

if new_altered_dataset.shape[0] > register_altered_new:
    list_index = [index for index, item in new_altered_dataset.iterrows()]

    list_index_insert_altered = []
    while len(list_index_insert_altered) < register_altered_new:
        aux = random.randint(0, new_altered_dataset.shape[0]-1)
        if aux in list_index:
            list_index_insert_altered.append(aux)
    
print('List of register altered dataset, %d register' % (len(list_index_insert_altered)))

In [ ]:
list_index_insert_aterm = [index for index, item in new_aterm_dataset.iterrows()]

if new_aterm_dataset.shape[0] > register_aterm_new:
    list_index = [index for index, item in new_aterm_dataset.iterrows()]

    list_index_insert_aterm = []
    while len(list_index_insert_aterm) < register_aterm_new:
        aux = random.randint(0, new_aterm_dataset.shape[0]-1)
        if aux in list_index:
            list_index_insert_aterm.append(aux)
    
print('List of register aterm dataset, %d register' % (len(list_index_insert_aterm)))

In [ ]:
new_dataset = new_altered_dataset

for i in list_index_insert_aterm:
    new_dataset = new_dataset.append(new_aterm_dataset.iloc[i])
    
print(new_dataset.shape[0])

In [ ]:
if len(columns_drop) > 0:
    new_dataset = new_dataset.drop(columns_drop, axis=1)

classify_name = "classify"
classify_col = new_dataset.pop(classify_name)
new_dataset.insert(new_dataset.shape[1], classify_name, classify_col)

for index, item in new_dataset.iterrows():
    if item['classify'] != 'Atermo':
        new_dataset['classify'][index] = 'Alterado'

In [ ]:
new_dataset.to_csv('dataset_final_treat.csv', index=False)